In [3]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI='sqlite:///mlflow.db'
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [7]:
client.create_experiment(name='my-cool-experiment')

'2'

In [16]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 5.3",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=['metrics.rmse ASC']
)

In [15]:
for run in runs:
    print(f'run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}')

run id: 70ac6f7b08ef42b3b4e7115810836c49, rmse: 5.1250
run id: 5c8593ac4f5245c58a4c1284c799ef99, rmse: 5.1250
run id: 7af79c48326e4d4d8a7dfab73c02992c, rmse: 5.1250
run id: 594fb116fd0f4636b187731d227b3b9e, rmse: 5.1543
run id: 3410751bed424d0b96c9f8931974387e, rmse: 105.1060


In [ ]:
70ac6f7b08ef42b3b4e7115810836c49

In [17]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [20]:
run_id='70ac6f7b08ef42b3b4e7115810836c49'
model_uri = f"run:/{run_id}/model"
mlflow.register_model(model_uri=model_uri,name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1743243097135, current_stage='None', description=None, last_updated_timestamp=1743243097135, name='nyc-taxi-regressor', run_id=None, run_link=None, source='run:/70ac6f7b08ef42b3b4e7115810836c49/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [23]:
model_name='nyc-taxi-regressor'
client.get_latest_versions(name=model_name)

C:\Users\rozak\AppData\Local\Temp\ipykernel_9624\2394194884.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.get_latest_versions(name=model_name)


[<ModelVersion: aliases=[], creation_timestamp=1743243097135, current_stage='None', description=None, last_updated_timestamp=1743243097135, name='nyc-taxi-regressor', run_id=None, run_link=None, source='run:/70ac6f7b08ef42b3b4e7115810836c49/model', status='READY', status_message=None, tags={}, user_id=None, version=3>]

In [26]:
model_name='nyc-taxi-regressor'
latest_versions=client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f'version: {version.version}, alias:{version.current_stage}')

version: 3, alias:None


C:\Users\rozak\AppData\Local\Temp\ipykernel_9624\3229042748.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions=client.get_latest_versions(name=model_name)
